In [2]:
import pandas as pd 
import numpy as np 
import scanpy as sc
import matplotlib.pyplot as plt
from datetime import date
import shutil
from adjustText import adjust_text
import warnings
import anndata
import gc
import os
import matplotlib as mpl
 
warnings.filterwarnings("ignore")
sc.settings.n_jobs = 60
sc._settings.settings._vector_friendly=True

print("Current working directory:", os.getcwd())

colors=pd.read_csv('/home/workspace/IHA_Figure_Revision/Color_hex_codes/Cell_Type_Color_Annotation_and_Order.csv')
color_dict=dict(zip(colors.label, colors.color))

/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/workspace/Aux_Envir/Python_BRI_Figures/lib/python3.11/site-p

Current working directory: /home/workspace/IHA_Figure_Revision/Extended_Figure7


In [3]:
def gen_mpl_labels(adata, groupby, exclude=(), ax=None, adjust_kwargs=None, text_kwargs=None, offset=(0.05, 0.05)):
    if adjust_kwargs is None:
        adjust_kwargs = {
            "arrowprops": {"arrowstyle": "->", "color": 'r'}, 
            "expand_points": (1.2, 1.2), 
            "force_points": 0.5
        }
    if text_kwargs is None:
        text_kwargs = {}

    highest_density_points = {}

    for g, g_idx in adata.obs.groupby(groupby).groups.items():
        if g in exclude:
            continue
        points = adata[g_idx].obsm["X_umap"]
        hist, xedges, yedges = np.histogram2d(points[:, 0], points[:, 1], bins=[50, 50])
        max_bin = np.unravel_index(np.argmax(hist), hist.shape)
        highest_density_point = [
            (xedges[max_bin[0]] + xedges[max_bin[0]+1])/2 + offset[0], 
            (yedges[max_bin[1]] + yedges[max_bin[1]+1])/2 + offset[1]
        ]
        highest_density_points[g] = highest_density_point

    if ax is None:
        ax = plt.gca()

    texts = [
        ax.text(x=x, y=y, s=k, **text_kwargs) for k, (x, y) in highest_density_points.items()
    ]

    adjust_text(texts, **adjust_kwargs)

In [4]:
adata=sc.read_h5ad("01_Burton_Dataset/adata_processed.h5ad")

In [5]:
selected_celltype = [
    'CD95 memory B cell',
    'CD27+ effector B cell',
    'Plasma cell',
    'CD27- effector B cell',
    'Core memory B cell'
]

with plt.rc_context({
    'figure.figsize': (60/25.4, 60/25.4),
    'figure.dpi': 500,
    'figure.frameon': False
}):
    adata.obs['AIFI_L3_Highlight'] = np.where(
        adata.obs['AIFI_L3'].isin(selected_celltype),
        adata.obs['AIFI_L3'],
        'other'
    )
    color_dict['other'] = '#a9a9a9'
    sc.pl.umap(
        adata,
        color='AIFI_L3',
        size=20,
        show=False,
        frameon=False,
        legend_loc=None
    )
    plt.savefig(
        "FigureG.pdf",
        format="pdf",
        bbox_inches="tight",
        dpi=1000
    )
    plt.close()